In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, json
import nltk

In [2]:
# Check GPU availability
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())

True
True


In [3]:
DATAPATH = 'data'

In [4]:
# load metadata
csv_filepath ="/".join([DATAPATH, 'metadata.csv'])
df = pd.read_csv(csv_filepath)

In [5]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389


In [6]:
# Check shape
df.shape

(45774, 17)

In [7]:
import os, os.path

DIR = 'data'
subdirectory = ['biorxiv_medrxiv', 'comm_use_subset', 'custom_license', 'noncomm_use_subset']

total_files = 0
for subdir in subdirectory:
    path = DIR + '/' + subdir + '/' + subdir
    print(subdir, len(os.listdir(path)))
    total_files += len(os.listdir(path))
total_files

biorxiv_medrxiv 1053
comm_use_subset 9315
custom_license 20657
noncomm_use_subset 2350


33375

In [8]:
print(df['full_text_file'].unique())
print(df['full_text_file'].shape)

['biorxiv_medrxiv' 'noncomm_use_subset' 'comm_use_subset' nan
 'custom_license']
(45774,)


In [9]:
print(df[df['full_text_file'].notna() == True]['full_text_file'].unique())
print(df[df['full_text_file'].notna() == True]['full_text_file'].shape)
df_sub = df[df['full_text_file'].notna() == True]

['biorxiv_medrxiv' 'noncomm_use_subset' 'comm_use_subset' 'custom_license']
(35558,)


In [10]:
total_files = 0
for subdir in subdirectory:
    df_temp = df[df['full_text_file'] == subdir]
    print(subdir, len(df_temp))
    total_files += len(df_temp)
total_files

biorxiv_medrxiv 1218
comm_use_subset 8982
custom_license 23227
noncomm_use_subset 2131


35558

In [11]:
# I checked, there are no duplicates is the full_text_file directories
total_files = []
for subdir in subdirectory:
    path = DIR + '/' + subdir + '/' + subdir
    print(subdir, len(os.listdir(path)))
    dir_set = set(os.listdir(path))
    print(subdir, len(dir_set))
    
    lst = [os.path.splitext(x)[0] for x in os.listdir(path)]
    total_files.extend(lst)
len(total_files)

biorxiv_medrxiv 1053
biorxiv_medrxiv 1053
comm_use_subset 9315
comm_use_subset 9315
custom_license 20657
custom_license 20657
noncomm_use_subset 2350
noncomm_use_subset 2350


33375

In [12]:
df_sub = df_sub[df_sub['sha'].isin(total_files)]
df_sub.shape

(30206, 17)

In [13]:
df_sub[df_sub['has_full_text'].notna() == True].shape

(30206, 17)

In [14]:
# These titles don't exist. These files don't exist. The sha points to a completely different file
dup_sha = set(df_sub['sha'][df_sub['sha'].duplicated()])

In [15]:
print(df_sub[df_sub['sha'].isin(dup_sha)].shape)
df_sub[df_sub['sha'].isin(dup_sha)]

(15, 17)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
1239,fjmchbew,58be092086c74c58e9067121a6ba4836468e7ec3,CZI,The Author's Response: Case of the Index Patie...,10.3346/jkms.2020.35.e89,NaN,32080993.0,cc-by-nc,NaN,2020,"Lim, Jaegyun; Jeon, Seunghyun; Shin, Hyun Youn...",J Korean Med Sci,3.005657e+09,#1576,True,noncomm_use_subset,https://doi.org/10.3346/jkms.2020.35.e89
27527,7gm6ovet,45e40b2d7d973ed5c9798da613fb3cfa4427e2e2,Elsevier,Vaccine biotechnology edited by James L. Bittl...,10.1016/0167-7799(89)90108-x,NaN,NaN,els-covid,NaN,1989-08-31,"Cavanagh, Dave",Trends in Biotechnology,NaN,NaN,True,custom_license,https://doi.org/10.1016/0167-7799(89)90108-x
27528,c75e60nm,45e40b2d7d973ed5c9798da613fb3cfa4427e2e2,Elsevier,Books received,10.1016/0167-7799(89)90109-1,NaN,NaN,els-covid,NaN,1989-08-31,NaN,Trends in Biotechnology,NaN,NaN,True,custom_license,https://doi.org/10.1016/0167-7799(89)90109-1
31107,d98q0k6r,f3aafdecdc43a3f57e58cf6dcea038b1834a953e,Elsevier,Primum Non Nocere,10.1378/chest.126.6.2026,NaN,15596712.0,els-covid,NaN,2004-12-31,"Sihoe, Alan D.L.; Wong, Randolph H.L.; Yim, An...",Chest,NaN,NaN,True,custom_license,https://doi.org/10.1378/chest.126.6.2026
31108,h46p2oj4,f3aafdecdc43a3f57e58cf6dcea038b1834a953e,Elsevier,Primum Non Nocere,10.1016/s0012-3692(15)31462-8,NaN,NaN,els-covid,NaN,2004-12-31,"Filice, Gregory A.",Chest,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0012-3692(15)31462-8
33296,mq8rwnqu,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,Coronavirus and gastroenteritis in foals,10.1016/s0140-6736(75)80058-4,NaN,78197.0,els-covid,NaN,1975-10-25,"Bass, E.P.; Sharpee, R.L.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80058-4
33297,fa21ckbr,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,Hypo-osmolality in beer drinkers,10.1016/s0140-6736(75)80059-6,NaN,78198.0,els-covid,NaN,1975-10-25,"Hilden, Tage; Svendsen, Tage Lysbo",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80059-6
33298,l2banbg8,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,A pamphlet to answer patients' questions,10.1016/s0140-6736(75)80060-2,NaN,78196.0,els-covid,NaN,1975-10-25,"Steele, S.J.; Goodwin, M.F.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80060-2
33324,qfyiu47z,ba4afe00e152de82121a4445aed52c46833d927e,Elsevier,FAILURE OF SALT TO MOBILISE RENAL DOPAMINE IN ...,10.1016/s0140-6736(80)92432-0,NaN,6109183.0,els-covid,NaN,1980-12-27,"Perkins, C.M.; Casson, I.F.; Cope, G.F.; Lee, ...",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(80)92432-0
33325,ouie3dp8,ba4afe00e152de82121a4445aed52c46833d927e,Elsevier,CORONAVIRUS-LIKE PARTICLES IN DIARRHOEA STOOLS,10.1016/s0140-6736(80)92433-2,NaN,6109182.0,els-covid,NaN,1980-12-27,"Curry, A.; Paver, W.K.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(80)92433-2


In [16]:
df_ft = df[df['has_full_text'] == True]
df_ft.shape

(31753, 17)

In [17]:
df_ft[df_ft['sha'].notna()==False].shape

(0, 17)

In [18]:
df_ft['sha'][df_ft['sha'].duplicated()]

27528    45e40b2d7d973ed5c9798da613fb3cfa4427e2e2
31108    f3aafdecdc43a3f57e58cf6dcea038b1834a953e
33297    9ce0a6cfd53840cd985f7a1439708c7a48bb7f23
33298    9ce0a6cfd53840cd985f7a1439708c7a48bb7f23
33325    ba4afe00e152de82121a4445aed52c46833d927e
41028    4644c32551fb23aa873a7738ecc8d777bd49877e
41029    4644c32551fb23aa873a7738ecc8d777bd49877e
41030    4644c32551fb23aa873a7738ecc8d777bd49877e
44847    58be092086c74c58e9067121a6ba4836468e7ec3
Name: sha, dtype: object

In [19]:
df_sub[df_sub['sha'].duplicated()]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
27528,c75e60nm,45e40b2d7d973ed5c9798da613fb3cfa4427e2e2,Elsevier,Books received,10.1016/0167-7799(89)90109-1,NaN,NaN,els-covid,NaN,1989-08-31,NaN,Trends in Biotechnology,NaN,NaN,True,custom_license,https://doi.org/10.1016/0167-7799(89)90109-1
31108,h46p2oj4,f3aafdecdc43a3f57e58cf6dcea038b1834a953e,Elsevier,Primum Non Nocere,10.1016/s0012-3692(15)31462-8,NaN,NaN,els-covid,NaN,2004-12-31,"Filice, Gregory A.",Chest,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0012-3692(15)31462-8
33297,fa21ckbr,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,Hypo-osmolality in beer drinkers,10.1016/s0140-6736(75)80059-6,NaN,78198.0,els-covid,NaN,1975-10-25,"Hilden, Tage; Svendsen, Tage Lysbo",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80059-6
33298,l2banbg8,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,A pamphlet to answer patients' questions,10.1016/s0140-6736(75)80060-2,NaN,78196.0,els-covid,NaN,1975-10-25,"Steele, S.J.; Goodwin, M.F.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80060-2
33325,ouie3dp8,ba4afe00e152de82121a4445aed52c46833d927e,Elsevier,CORONAVIRUS-LIKE PARTICLES IN DIARRHOEA STOOLS,10.1016/s0140-6736(80)92433-2,NaN,6109182.0,els-covid,NaN,1980-12-27,"Curry, A.; Paver, W.K.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(80)92433-2
41028,bcn6tpnx,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-34 A fast procedure for the detection of t...,10.1016/s1386-6532(09)70130-1,NaN,NaN,els-covid,NaN,2009-09-30,"Allice, V.; Milia, M.G.; Gregori, G.; Proia, M...",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70130-1
41029,n8n1folf,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-35 Evaluation of two newly developed QIAsy...,10.1016/s1386-6532(09)70131-3,NaN,NaN,els-covid,NaN,2009-09-30,"Hanselle, T.; Kasper, Y.; Otte, M.; Ritter, S....",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70131-3
41030,jof3mnnk,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-36 Performance of the Qiagen Resplex II ve...,10.1016/s1386-6532(09)70132-5,NaN,NaN,els-covid,NaN,2009-09-30,"Halligan, E.; Bible, J.; Sehmi, J.; Dunn, H.; ...",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70132-5
44847,u4taazr3,58be092086c74c58e9067121a6ba4836468e7ec3,PMC,Letter to the Editor: Case of the Index Patien...,10.3346/jkms.2020.35.e88,PMC7036343,32080992.0,cc-by-nc,NaN,2020-02-20,"Kim, Jin Yong",J Korean Med Sci,NaN,#1608,True,noncomm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...


In [20]:
df_sub[df_sub['abstract'] != 'Unknown']

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45764,xupeqtx3,a186e1e74616d4936c8de93c42a857c4cb9d1edf,Elsevier,One-tube smart genetic testing via coupling is...,10.1016/j.aca.2020.01.068,PMC7094703,32145848.0,els-covid,Abstract Urgent demand for portable diagnosis ...,2020-04-15,"Guo, Lulu; Lu, Baiyang; Dong, Qing; Tang, Yida...",Analytica Chimica Acta,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.aca.2020.01.068
45765,f6mlvl56,efd9f0bbc3ac52b299b2799aa8d72cd9a5b55ccf,Elsevier,Sumoylation of the nucleocapsid protein of sev...,10.1016/j.febslet.2005.03.039,PMC7094623,15848177.0,els-covid,Abstract Severe acute respiratory syndrome cor...,2005-04-25,"Li, Frank Qisheng; Xiao, Han; Tam, James P.; L...",FEBS Letters,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.febslet.2005.03.039
45768,6b1y7yxg,f81692543d3e35858911cea48c298bfa23b20bc6,Elsevier,Quality of life and psychological status in su...,10.1016/j.jpsychores.2005.08.020,PMC7094294,16650592.0,els-covid,Abstract Background Little is known about the ...,2006-05-31,"Kwek, Seow-Khee; Chew, Wuen-Ming; Ong, Kian-Ch...",Journal of Psychosomatic Research,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.jpsychores.2005.08.020
45769,4360s2yu,289deae0b2050aa259a05ba84565a4df82fa099a,Elsevier,Personal Protective Equipment: Protecting Heal...,10.1016/j.clinthera.2015.07.007,PMC4661082,26452427.0,els-covid,Abstract Purpose The recent Ebola epidemic tha...,2015-11-01,"Fischer, William A.; Weber, David J.; Wohl, Da...",Clinical Therapeutics,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.clinthera.2015.07.007


In [21]:
df_sub = df_sub[df_sub['abstract'].notna() == True]
df_sub.shape

(26450, 17)

In [36]:
df_sub[df['abstract'].str.len() >= 500]

C:\Users\Khachatur\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45764,xupeqtx3,a186e1e74616d4936c8de93c42a857c4cb9d1edf,Elsevier,One-tube smart genetic testing via coupling is...,10.1016/j.aca.2020.01.068,PMC7094703,32145848.0,els-covid,Abstract Urgent demand for portable diagnosis ...,2020-04-15,"Guo, Lulu; Lu, Baiyang; Dong, Qing; Tang, Yida...",Analytica Chimica Acta,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.aca.2020.01.068
45765,f6mlvl56,efd9f0bbc3ac52b299b2799aa8d72cd9a5b55ccf,Elsevier,Sumoylation of the nucleocapsid protein of sev...,10.1016/j.febslet.2005.03.039,PMC7094623,15848177.0,els-covid,Abstract Severe acute respiratory syndrome cor...,2005-04-25,"Li, Frank Qisheng; Xiao, Han; Tam, James P.; L...",FEBS Letters,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.febslet.2005.03.039
45768,6b1y7yxg,f81692543d3e35858911cea48c298bfa23b20bc6,Elsevier,Quality of life and psychological status in su...,10.1016/j.jpsychores.2005.08.020,PMC7094294,16650592.0,els-covid,Abstract Background Little is known about the ...,2006-05-31,"Kwek, Seow-Khee; Chew, Wuen-Ming; Ong, Kian-Ch...",Journal of Psychosomatic Research,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.jpsychores.2005.08.020
45769,4360s2yu,289deae0b2050aa259a05ba84565a4df82fa099a,Elsevier,Personal Protective Equipment: Protecting Heal...,10.1016/j.clinthera.2015.07.007,PMC4661082,26452427.0,els-covid,Abstract Purpose The recent Ebola epidemic tha...,2015-11-01,"Fischer, William A.; Weber, David J.; Wohl, Da...",Clinical Therapeutics,NaN,NaN,True,custom_license,https://doi.org/10.1016/j.clinthera.2015.07.007


In [22]:
import numpy as np
def extract_docs(df):
    docs = []
    for row in df.iterrows():
        abstract = row[1]['abstract']
        title = row[1]['title']
        if len(abstract) < 500 :
            continue
        if len(title) < 20:
            title = ''
        doc = title + ' ' + abstract
        docs.append(doc)
    
    return docs

In [23]:
docs = extract_docs(df_sub)

In [24]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
# Split the documents into tokens.
def simple_preprocess(docs):
    new_docs = []
    stop_words = set(stopwords.words('english'))
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        new_doc = docs[idx].lower()  # Convert to lowercase.
        new_doc = tokenizer.tokenize(new_doc)  # Split into words.
        new_doc = [token for token in new_doc if not token.isnumeric()]
        new_doc = [token for token in new_doc if len(token) > 1]
        new_doc = [token for token in new_doc if token not in stop_words]
        new_docs.append(new_doc)
    
    return new_docs

In [25]:
docs = simple_preprocess(docs)
print(docs)

['siann', 'strain', 'identification', 'alignment', 'near', 'neighbors', 'next', 'generation', 'sequencing', 'increasingly', 'used', 'study', 'samples', 'composed', 'mixtures', 'organisms', 'clinical', 'applications', 'presence', 'pathogen', 'low', 'abundance', 'may', 'highly', 'important', 'present', 'analytical', 'method', 'siann', 'strain', 'identification', 'alignment', 'near', 'neighbors', 'specifically', 'designed', 'rapidly', 'detect', 'set', 'target', 'organisms', 'mixed', 'samples', 'achieves', 'high', 'degree', 'species', 'strain', 'specificity', 'aligning', 'short', 'sequence', 'reads', 'genomes', 'near', 'neighbor', 'organisms', 'well', 'target', 'empirical', 'benchmarking', 'alongside', 'current', 'state', 'art', 'methods', 'shows', 'extremely', 'high', 'positive', 'predictive', 'value', 'even', 'low', 'abundances', 'target', 'organism', 'mixed', 'sample', 'siann', 'available', 'illumina', 'basespace', 'app', 'well', 'signature', 'science', 'llc', 'siann', 'results', 'prese

In [26]:
test = docs[0].copy()
print(test)

['siann', 'strain', 'identification', 'alignment', 'near', 'neighbors', 'next', 'generation', 'sequencing', 'increasingly', 'used', 'study', 'samples', 'composed', 'mixtures', 'organisms', 'clinical', 'applications', 'presence', 'pathogen', 'low', 'abundance', 'may', 'highly', 'important', 'present', 'analytical', 'method', 'siann', 'strain', 'identification', 'alignment', 'near', 'neighbors', 'specifically', 'designed', 'rapidly', 'detect', 'set', 'target', 'organisms', 'mixed', 'samples', 'achieves', 'high', 'degree', 'species', 'strain', 'specificity', 'aligning', 'short', 'sequence', 'reads', 'genomes', 'near', 'neighbor', 'organisms', 'well', 'target', 'empirical', 'benchmarking', 'alongside', 'current', 'state', 'art', 'methods', 'shows', 'extremely', 'high', 'positive', 'predictive', 'value', 'even', 'low', 'abundances', 'target', 'organism', 'mixed', 'sample', 'siann', 'available', 'illumina', 'basespace', 'app', 'well', 'signature', 'science', 'llc', 'siann', 'results', 'prese

In [27]:
# stop_words = set(stopwords.words('english'))
# test = [token for token in test if token not in stop_words]
# test

In [28]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

In [29]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [30]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 9691
Number of documents: 25339


In [31]:
from gensim.models import LdaMulticore

# Set training parameters.
passes = 10
num_topics = 50

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaMulticore(
    corpus=corpus,
    id2word=id2word,
    passes=passes,
)

In [34]:
model.save('models/lda_abs_bench/model')

In [32]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

Average topic coherence: -7.7278.


In [33]:
top_topics

[([(0.17219701, 'sars'),
   (0.103164285, 'cov'),
   (0.04944565, 'coronavirus'),
   (0.028236307, 'severe'),
   (0.026713898, 'syndrome'),
   (0.026505353, 'respiratory'),
   (0.026134457, 'acute'),
   (0.013323976, 'ncov'),
   (0.010533419, 'covid'),
   (0.009604525, 'protein'),
   (0.009057052, 'coronaviruses'),
   (0.008933498, 'outbreak'),
   (0.006668962, 'novel'),
   (0.0058262562, 'human'),
   (0.0054191817, 'caused'),
   (0.0050999713, 'patients'),
   (0.0050747264, 'spike'),
   (0.0047445195, 'china'),
   (0.004532015, 'viral'),
   (0.004499592, 'infection')],
  -1.6180269246196264),
 ([(0.048844185, 'respiratory'),
   (0.02267667, 'viruses'),
   (0.021575209, 'patients'),
   (0.01925472, 'pcr'),
   (0.01590612, 'virus'),
   (0.0154769, 'samples'),
   (0.014946337, 'positive'),
   (0.013207083, 'infections'),
   (0.012990967, 'detected'),
   (0.011793888, 'viral'),
   (0.010941635, 'study'),
   (0.010658231, 'results'),
   (0.009734438, 'detection'),
   (0.009425858, 'influen

In [35]:
len(docs)

25339